<a href="https://colab.research.google.com/github/sarkar-sayan/Log-Ingestor-and-Query-Interface/blob/main/POC_1_URL_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split # function for splitting data to train and test sets

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier

from bs4 import BeautifulSoup
import requests
import time
from random import choice
from urllib.parse import urlparse

from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# List of user-agents for rotation
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0',
    # Add more user-agents if needed
]

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Sayan RP files/Datasets/URL_Dataset(Sheet1).csv")
df.head()

,index,url,label
0,1,https://stackoverflow.com/questions/66341659/e...,Productive
1,2,https://www.anandabazar.com/west-bengal/state-...,Non-productive
2,3,https://www.barandbench.com/news/sim-swap-scam...,Non-productive
3,4,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive
4,5,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive


In [5]:
df.columns

Index(['index', 'url', 'label'], dtype='object')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   index   5 non-null      int64 
 1   url     5 non-null      object
 2   label   5 non-null      object
dtypes: int64(1), object(2)
memory usage: 248.0+ bytes


In [7]:
cols = df.columns
cols

Index(['index', 'url', 'label'], dtype='object')

In [8]:
df.drop(['index'],axis=1,inplace=True)

In [9]:
df

,url,label
0,https://stackoverflow.com/questions/66341659/e...,Productive
1,https://www.anandabazar.com/west-bengal/state-...,Non-productive
2,https://www.barandbench.com/news/sim-swap-scam...,Non-productive
3,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive
4,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive


In [10]:
df['label'].unique()

array(['Productive', 'Non-productive'], dtype=object)

In [11]:
df[df.label == 'Non-productive'].head(10)

,url,label
1,https://www.anandabazar.com/west-bengal/state-...,Non-productive
2,https://www.barandbench.com/news/sim-swap-scam...,Non-productive
3,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive


In [12]:
df[df.label == 'Productive'].head(10)

,url,label
0,https://stackoverflow.com/questions/66341659/e...,Productive
4,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive


In [13]:
# label_encoder object knows how to understand word labels.                 FOR LABEL ENCODER IF NEEDED
#label_encoder = preprocessing.LabelEncoder()
#df['label']= label_encoder.fit_transform(df['label'])

#df['label'].unique()

In [14]:
#df[df.label == 1].head(10)  #label for Productive -> 1

In [15]:
#df[df.label == 0].head(10)  #label for Non-productive -> 0

In [16]:
# Function to get text and metadata from a URL
def get_metadata_from_url(url):
    try:
        headers = {'User-Agent': choice(user_agents)}
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract the title
        title = soup.find('title').get_text() if soup.find('title') else 'No title'

        # Extract the description
        description = None
        if soup.find('meta', attrs={'name': 'description'}):
            description = soup.find('meta', attrs={'name': 'description'}).get('content')
        elif soup.find('meta', attrs={'property': 'og:description'}):
            description = soup.find('meta', attrs={'property': 'og:description'}).get('content')
        else:
            description = 'No description'

        # Extract the image URL
        image = None
        if soup.find('meta', attrs={'property': 'og:image'}):
            image = soup.find('meta', attrs={'property': 'og:image'}).get('content')
        elif soup.find('link', attrs={'rel': 'image_src'}):
            image = soup.find('link', attrs={'rel': 'image_src'}).get('href')
        else:
            image = 'No image'

        # Extract the main content
        text = ' '.join(p.get_text() for p in soup.find_all('p'))

        return {
            "title": title,
            "description": description,
            "image": image,
            "url": url,
            "text": text
        }
    except requests.RequestException as e:
        return {
            "title": "Error",
            "description": str(e),
            "image": "No image",
            "url": url,
            "text": ""
        }

In [17]:
# Step 2: Scrape Content and Metadata
df['metadata'] = df['url'].apply(get_metadata_from_url)

In [18]:
df.head()

,url,label,metadata
0,https://stackoverflow.com/questions/66341659/e...,Productive,{'title': 'uncaught exception - Electron ipcMa...
1,https://www.anandabazar.com/west-bengal/state-...,Non-productive,"{'title': 'Error', 'description': '403 Client ..."
2,https://www.barandbench.com/news/sim-swap-scam...,Non-productive,{'title': 'SIM swap scam: NCDRC restores order...
3,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive,{'title': 'East Bengal FC 0 - 1 Mohun Bagan SG...
4,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive,{'title': 'Collections in Python - Advanced Py...


In [19]:
df['metadata'][4]

{'title': 'Collections in Python - Advanced Python 06 - Programming Tutorial - YouTube',
 'description': 'Collections in Python - Advanced Python 06 - Programming TutorialIn this Python Advanced Tutorial, we will be learning about the collections module in Python...',
 'image': 'https://i.ytimg.com/vi/UdcPhnNjSEw/maxresdefault.jpg',
 'url': 'https://www.youtube.com/watch?v=UdcPhnNjSEw',
 'text': ''}

In [20]:
# Step 3: Preprocess Text
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    if not text:
        return ""
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Preprocess metadata function
def preprocess_metadata(metadata):
    # Preprocess each field in the metadata
    clean_title = preprocess_text(metadata.get('title', ''))
    clean_description = preprocess_text(metadata.get('description', ''))
    clean_text = preprocess_text(metadata.get('text', ''))

    # Combine cleaned fields
    combined_clean_content = f"{clean_title} {clean_description} {clean_text}".strip()

    return combined_clean_content

# Apply preprocessing to metadata
df['clean_content'] = df['metadata'].apply(preprocess_metadata)

In [21]:
df['clean_content']

0    uncaught exception electron ipcmain gracefully...
1    error 403 client error forbidden url httpswwwa...
2    sim swap scam ncdrc restores order directing a...
3    east bengal fc 0 1 mohun bagan sg final highli...
4    collections python advanced python 06 programm...
Name: clean_content, dtype: object

In [22]:
df

,url,label,metadata,clean_content
0,https://stackoverflow.com/questions/66341659/e...,Productive,{'title': 'uncaught exception - Electron ipcMa...,uncaught exception electron ipcmain gracefully...
1,https://www.anandabazar.com/west-bengal/state-...,Non-productive,"{'title': 'Error', 'description': '403 Client ...",error 403 client error forbidden url httpswwwa...
2,https://www.barandbench.com/news/sim-swap-scam...,Non-productive,{'title': 'SIM swap scam: NCDRC restores order...,sim swap scam ncdrc restores order directing a...
3,https://www.youtube.com/watch?v=D_qLCwDiRs4\u0...,Non-productive,{'title': 'East Bengal FC 0 - 1 Mohun Bagan SG...,east bengal fc 0 1 mohun bagan sg final highli...
4,https://www.youtube.com/watch?v=UdcPhnNjSEw,Productive,{'title': 'Collections in Python - Advanced Py...,collections python advanced python 06 programm...


In [23]:
# Step 4: Feature Extraction
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['clean_content'])
y = df['label']

    # Verify Preprocessed Data
#print(df['clean_content'].head())
    # Check Data Distribution
#print(df['label'].value_counts())

# Step 5: Train Model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = MultinomialNB()
model.fit(X_train, y_train)
#print(f"Training samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

# Step 6: Evaluate Model
y_pred = model.predict(X_test)
#print("Accuracy:", accuracy_score(y_test, y_pred))
#print(classification_report(y_test, y_pred))

# Step 7: Predict
def classify_url(url):
    metadata = get_metadata_from_url(url)
    clean_content = preprocess_text(metadata['text'])
    vectorized_content = vectorizer.transform([clean_content])
    return model.predict(vectorized_content)[0]

In [24]:
# Example Usage
new_url = "https://stackoverflow.com/questions/66341659/electron-ipcmain-how-to-gracefully-handle-throwing-an-error"
print(f"The URL {new_url} is classified as {classify_url(new_url)}")

The URL https://stackoverflow.com/questions/66341659/electron-ipcmain-how-to-gracefully-handle-throwing-an-error is classified as Productive


In [25]:
new_url = "https://www.anandabazar.com/video/exclusive-interview-of-cpm-leader-goutam-deb-in-anandabazar-online-dgtl/cid/1520839"
print(f"The URL {new_url} is classified as {classify_url(new_url)}")

The URL https://www.anandabazar.com/video/exclusive-interview-of-cpm-leader-goutam-deb-in-anandabazar-online-dgtl/cid/1520839 is classified as Non-productive


In [26]:
new_url = "https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/"
print(f"The URL {new_url} is classified as {classify_url(new_url)}")

The URL https://www.geeksforgeeks.org/ml-label-encoding-of-datasets-in-python/ is classified as Productive


In [27]:
new_url = input("Enter url to check for Productive or Non-productive: ")
print(f"The URL {new_url} is classified as {classify_url(new_url)}")

Enter url to check for Productive or Non-productive: https://www.youtube.com/watch?v=th5_9woFJmk
The URL https://www.youtube.com/watch?v=th5_9woFJmk is classified as Non-productive
